<a href="https://colab.research.google.com/github/rogiafarias/IAtalentSquad/blob/main/Projeto_IA_TalentSquad_junho2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IA Talent Squad - Ferramenta de análise de documentos fiscais








# Etapa 1 Instalação de Dependências e Importações

In [5]:
pip install ag2

# Etapa 2: Importações e Configuração Inicial

In [6]:

import os
import zipfile
import requests
import pandas as pd
from autogen import AssistantAgent, UserProxyAgent
from IPython.display import display, Markdown
import ipywidgets as widgets
from google.colab import userdata
import traceback

# --- CONFIGURAÇÃO ---


In [7]:
# Configurar variáveis de ambiente
ENDPOINT_API = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateText?key="
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# Etapa 3: Função Otimizada para Preparação dos Dados

In [8]:
import pandas as pd
# ✅ Carregar dados diretamente da pasta extraída
cabecalho = pd.read_csv('/content/202401_NFs_Cabecalho.csv')
itens = pd.read_csv('/content/202401_NFs_Itens.csv')

cabecalho_df = cabecalho
itens_df = itens

# Etapa 4: Configuração do Agente AutoGen



In [9]:
#O contexto (schema dos dados) é passado na mensagem de sistema para o agente "nascer"
import os
import autogen
from autogen import AssistantAgent, UserProxyAgent
import pandas as pd
CONTEXTO_SISTEMA = f"""
Você é um analista de dados especialista em notas fiscais, preciso e direto.
Sua tarefa é responder às perguntas do usuário usando os DataFrames do Pandas fornecid
Os DataFrames disponíveis são `cabecalho_df` e `itens_df`.
Schema do `cabecalho_df`: {list(cabecalho_df.columns)}
Schema do `itens_df`: {list(itens_df.columns)}
Para responder, você deve:
1. Analisar a pergunta do usuário.
2. Gerar e executar o código Python necessário para encontrar a resposta.
3. Apresentar a resposta final em linguagem natural, de forma clara e objetiva.
3. Apresentar a resposta final em linguagem natural, de forma clara e objetiva.
4. Não mostre o código Python na sua resposta final, apenas o resultado.
"""
#O contexto (schema dos dados) é passado na mensagem de sistema para o agente "nascer"
import os
import autogen
from autogen import AssistantAgent, UserProxyAgent
import pandas as pd
CONTEXTO_SISTEMA = f"""
Você é um analista de dados especialista em notas fiscais, preciso e direto.
Sua tarefa é responder às perguntas do usuário usando os DataFrames do Pandas fornecid
Os DataFrames disponíveis são `cabecalho_df` e `itens_df`.
Schema do `cabecalho_df`: {list(cabecalho_df.columns)}
Schema do `itens_df`: {list(itens_df.columns)}
Para responder, você deve:
1. Analisar a pergunta do usuário.
2. Gerar e executar o código Python necessário para encontrar a resposta.
3. Apresentar a resposta final em linguagem natural, de forma clara e objetiva.
4. Não mostre o código Python na sua resposta final, apenas o resultado.
"""

# Etapa 5: Interface Comunicacional

In [10]:
# Widgets para interface
pergunta_input = widgets.Text(placeholder="Ex: Qual o valor total vendido?")
botao_enviar = widgets.Button(description="Analisar")
saida = widgets.Output()
def execute_code(code, local_vars=None):
    """
    Executa código Python de forma segura e captura erros.
    """
    if local_vars is None:
        local_vars = {}
    try:
        exec(code, globals(), local_vars) # Execute no escopo local
        return local_vars.get('result', None), None # Retorna 'result' se existir
    except Exception as e:
        return None, traceback.format_exc() # Retorna None e o traceback do erro

def on_button_click(b):
    """
    Função chamada quando o botão é clicado.
    Limpa a saída anterior, inicia o chat com o agente e exibe a resposta final.
    """
    pergunta = pergunta_input.value
    if not pergunta:
        with saida:
            saida.clear_output()
            print("Por favor, digite uma pergunta.")
        return

    with saida:
        saida.clear_output()
        print(f"Processando a pergunta: '{pergunta}'...")

        # Inicia o chat. O prompt é mais completo, incluindo informações dos DataFrame
        prompt = f"""
        Dados disponíveis:
        1. cabecalho_df: {cabecalho_df.head(2).to_dict()}
        2. itens_df: {itens_df.head(2).to_dict()}

        Pergunta do usuário: {pergunta}

        Instruções:
        1. Analise os dataframes 'cabecalho_df' e 'itens_df'.
        2. Gere código Python para responder à pergunta do usuário. O resultado deve ser atribuído a uma variável chamada 'result'.
        3. Use apenas as informações fornecidas e pandas para manipulação dos dados.
        4. Execute o código Python gerado.
        5. Retorne a resposta formatada em português, incluindo o valor da variável 'result'.
        6. Inclua a pergunta original na resposta.
        7. Se não for possível responder com os dados, explique o motivo.
        """

        user_proxy.initiate_chat(
            assistant,
            message=prompt
        )

        # Captura a resposta final de forma robusta, pegando a última mensagem da conversa
        resposta_completa = user_proxy.last_message().get("content", "Não foi possível obter a resposta completa.")

        try:
            # Tenta extrair o código Python da resposta
            # Assumindo que o LLM envolve o código em ```python ... ```
            codigo_inicio = resposta_completa.find("```python") + len("```python\n")
            codigo_fim = resposta_completa.find("```", codigo_inicio)
            codigo = resposta_completa[codigo_inicio:codigo_fim].strip()

            # Executa o código extraído
            if codigo:
                result, erro = execute_code(codigo, local_vars={'cabecalho_df': cabecalho_df, 'itens_df': itens_df})
                if erro:
                    resposta_final = f"Ocorreu um erro ao executar o código: {erro}"
                else:
                    resposta_final = f"Resultado da análise: {result}" if result is not None else "Não foi possível determinar o resultado da análise."
            else:
                resposta_final = "Não foi possível extrair o código Python da resposta do agente."

        except Exception as e:
            resposta_final = f"Ocorreu um erro ao processar a resposta: {traceback.format_exc()}"

        # Exibe a resposta formatada em Markdown
        display(Markdown(f"### ✨ Análise Solicitada\n**Pergunta:** {pergunta}\n\n**Resposta:** {resposta_final}"))

# Associa a função ao evento de clique do botão
botao_enviar.on_click(on_button_click)

# Exibe a interface final
print("Interface pronta. Qual sua pergunta ao IA Talent Squad?.")
display(widgets.VBox([pergunta_input, botao_enviar, saida]))

Interface pronta. Qual sua pergunta ao IA Talent Squad?.
